<a href="https://colab.research.google.com/github/OnroerendErfgoed/scriptorium/blob/main/notebooks/12_generate_a_thesaurus_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating a thesaurus tree

For this script we use the skosprovider_atramhasis library to have easier access to the [Flanders Heritage thesauri](https://thesaurus.onroerenderfgoed).

In [ ]:
!pip install skosprovider_atramhasis

In [25]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
'''
This script demonstrates using the AtramhasisProvider connected to https://thesaurus.onroerenderfgoed.be/conceptschemes/ERFGOEDTYPES to print a thesaurus or main branches thereof as a simple rst tree.

Optionally, it can query the Flanders Heritage Inventory (https://inventaris.onroerenderfgoed.be) to report on the number of times a certain concept is assigned to erfgoed- or aanduidingsobjecten.
'''
import requests
from IPython.display import display
from IPython.display import Markdown

from skosprovider_atramhasis.providers import AtramhasisProvider
from skosprovider.providers import DictionaryProvider
from skosprovider.utils import dict_dumper

# Print the entire thesaurus
#TOP_CONCEPTS = []
# Use to only generate the tree for certain top terms.
# TOP_CONCEPTS = [147, 237, 930, 933, 78, 1751]
# Only print the tree for RELIGIEUZE GEBOUWEN EN COMPLEXEN
TOP_CONCEPTS = [359]

# Query the FHA inventory for the number of times these are used?
QUERY_USAGE= True

# Which language should the thesaurus be displayed in?
# (This will generally not provide a lot of results for anything other than nl or nl-BE)
LANGUAGE = 'nl-BE'

INVENTARIS_HOST = 'https://inventaris.onroerenderfgoed.be/'
URL_ERFGOEDOBJECTEN = INVENTARIS_HOST + 'erfgoedobjecten'
URL_AANDUIDINGSOBJECTEN = INVENTARIS_HOST + 'aanduidingsobjecten'

def find_aantal_erfgoedobjecten(concept_id):

    res = requests.get(
        URL_ERFGOEDOBJECTEN,
        params={'typologie': concept_id},
        headers={'accept': 'application/json'}
    )
    aantal = None
    if res.status_code == requests.codes.ok:
        if 'Content-Range' in res.headers:
            aantal = res.headers['Content-Range'].split('/')[-1]
        else:
            aantal = 0
    return (aantal, res.url)

def find_aantal_aanduidingsobjecten(concept_id):
    res = requests.get(
        URL_AANDUIDINGSOBJECTEN,
        params={
            'typologie': concept_id,
            'geldig': 1,
            'categorie': 'beschermingen'
        },
        headers={'accept': 'application/json'}
    )
    aantal = None
    if res.status_code == requests.codes.ok:
        if 'Content-Range' in res.headers:
            aantal = res.headers['Content-Range'].split('/')[-1]
        else:
            aantal = 0
    return (aantal, res.url)

def list_concepts(items, provider, indent=''):
    output = ''
    for i in items:
        if i['type'] == 'collection':
          output += f"{indent}* <[{i['label']}]({i['uri']})>\n"
        else:
            if QUERY_USAGE:
                erfgoedobjecten = find_aantal_erfgoedobjecten(i['id'])
                aanduidingsobjecten = find_aantal_aanduidingsobjecten(i['id'])
                output += f"{indent}* [{i['label']}]({i['uri']}) ([{erfgoedobjecten[0]}]({erfgoedobjecten[1]}) erfgoedobjecten, [{aanduidingsobjecten[0]}]({aanduidingsobjecten[1]}) aanduidingsobjecten)\n"
            else:
              output += f"{indent}* [{i['label']}]({i['uri']})\n"

        child = provider.get_children_display(
            i['id'],
            language=LANGUAGE,
            sort='label'
        )
        if (len(child)):
            output += list_concepts(child, provider, indent=indent + ' ')
    return output

def main():
    # Keep cache in between runs of the script
    # Value is considered valid for 1 day
    provider = AtramhasisProvider(
        {
            'id': 'vioe-erfgoedtypes)',
            'uri': 'https://id.erfgoed.net/thesauri/erfgoedtypes'
        },
        base_url='https://thesaurus.onroerenderfgoed.be',
        scheme_id='ERFGOEDTYPES',
        cache_config={
            'cache.backend': 'dogpile.cache.dbm',
            'cache.expiration_time': 60 * 60 * 24,
            'cache.arguments.filename': 'erfgoedtypes.dbm'
        }
    )

    thesaurus_title = provider.concept_scheme.label(LANGUAGE).label
    output = f'#{thesaurus_title}\n'

    # Fetch only certain top concepts or fetch the entire thesaurus
    if len(TOP_CONCEPTS):
        top = []
        for tcid in TOP_CONCEPTS:
            tc = provider.get_by_id(tcid)
            top.append({
                'id': tc.id,
                'uri': tc.uri,
                'type': tc.type,
                'label': tc.label(LANGUAGE).label
            })
    else:
        top = provider.get_top_display(LANGUAGE,sort='label')

    # For each top concept, generate the subtree
    for t in top:
        if t['type'] == 'collection':
          output += f"##<{t['label']}> ({t['uri']})\n"
        else:
          output += f"##{t['label']} ({t['uri']})\n"

        child = provider.get_children_display(
            t['id'],
            language='nl-BE',
            sort='label'
        )
        output += list_concepts(child, provider)
    
    display(Markdown(output))

if __name__ == "__main__":
  main()

#Erfgoedtypes
##religieuze gebouwen en complexen (https://id.erfgoed.net/thesauri/erfgoedtypes/359)
* [kerken](https://id.erfgoed.net/thesauri/erfgoedtypes/1005) ([2054](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=1005) erfgoedobjecten, [1155](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=1005&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [katholieke kerken](https://id.erfgoed.net/thesauri/erfgoedtypes/230) ([2005](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=230) erfgoedobjecten, [1145](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=230&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [kloosterkerken](https://id.erfgoed.net/thesauri/erfgoedtypes/242) ([120](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=242) erfgoedobjecten, [84](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=242&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [bedevaartkerken](https://id.erfgoed.net/thesauri/erfgoedtypes/47) ([33](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=47) erfgoedobjecten, [18](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=47&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [noodkerken](https://id.erfgoed.net/thesauri/erfgoedtypes/308) ([20](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=308) erfgoedobjecten, [1](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=308&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [kathedralen](https://id.erfgoed.net/thesauri/erfgoedtypes/226) ([7](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=226) erfgoedobjecten, [8](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=226&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [kapittelkerken](https://id.erfgoed.net/thesauri/erfgoedtypes/218) ([29](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=218) erfgoedobjecten, [22](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=218&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [basilieken](https://id.erfgoed.net/thesauri/erfgoedtypes/772) ([8](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=772) erfgoedobjecten, [8](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=772&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [parochiekerken](https://id.erfgoed.net/thesauri/erfgoedtypes/338) ([1792](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=338) erfgoedobjecten, [992](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=338&geldig=1&categorie=beschermingen) aanduidingsobjecten)
   * [hulpkerken](https://id.erfgoed.net/thesauri/erfgoedtypes/720) ([9](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=720) erfgoedobjecten, [0](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=720&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [begijnhofkerken](https://id.erfgoed.net/thesauri/erfgoedtypes/52) ([21](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=52) erfgoedobjecten, [25](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=52&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [abdijkerken](https://id.erfgoed.net/thesauri/erfgoedtypes/6) ([43](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=6) erfgoedobjecten, [35](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=6&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [protestantse kerken](https://id.erfgoed.net/thesauri/erfgoedtypes/355) ([17](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=355) erfgoedobjecten, [11](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=355&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [orthodoxe kerken](https://id.erfgoed.net/thesauri/erfgoedtypes/330) ([2](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=330) erfgoedobjecten, [1](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=330&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [moskeeën](https://id.erfgoed.net/thesauri/erfgoedtypes/298) ([0](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=298) erfgoedobjecten, [0](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=298&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [begijnhoven](https://id.erfgoed.net/thesauri/erfgoedtypes/53) ([28](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=53) erfgoedobjecten, [44](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=53&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [kapellen (gebouwen en structuren)](https://id.erfgoed.net/thesauri/erfgoedtypes/513) ([1473](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=513) erfgoedobjecten, [638](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=513&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [bedevaartkapellen](https://id.erfgoed.net/thesauri/erfgoedtypes/46) ([143](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=46) erfgoedobjecten, [106](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=46&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [kerkhofkapellen](https://id.erfgoed.net/thesauri/erfgoedtypes/231) ([36](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=231) erfgoedobjecten, [14](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=231&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [kloosters](https://id.erfgoed.net/thesauri/erfgoedtypes/243) ([1007](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=243) erfgoedobjecten, [299](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=243&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [kluizen](https://id.erfgoed.net/thesauri/erfgoedtypes/244) ([18](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=244) erfgoedobjecten, [8](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=244&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [cultusplaatsen](https://id.erfgoed.net/thesauri/erfgoedtypes/1116) ([0](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=1116) erfgoedobjecten, [1](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=1116&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [landelijke heiligdommen](https://id.erfgoed.net/thesauri/erfgoedtypes/1117) ([0](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=1117) erfgoedobjecten, [0](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=1117&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [tempels (archeologisch erfgoed)](https://id.erfgoed.net/thesauri/erfgoedtypes/1119) ([0](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=1119) erfgoedobjecten, [1](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=1119&geldig=1&categorie=beschermingen) aanduidingsobjecten)
  * [inheems-Romeinse tempels](https://id.erfgoed.net/thesauri/erfgoedtypes/1120) ([0](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=1120) erfgoedobjecten, [0](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=1120&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [viereckschanzen](https://id.erfgoed.net/thesauri/erfgoedtypes/1121) ([0](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=1121) erfgoedobjecten, [0](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=1121&geldig=1&categorie=beschermingen) aanduidingsobjecten)
 * [mithraea](https://id.erfgoed.net/thesauri/erfgoedtypes/1118) ([0](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=1118) erfgoedobjecten, [0](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=1118&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [synagogen](https://id.erfgoed.net/thesauri/erfgoedtypes/420) ([8](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=420) erfgoedobjecten, [4](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=420&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [vrijmetselaarstempels](https://id.erfgoed.net/thesauri/erfgoedtypes/480) ([4](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=480) erfgoedobjecten, [4](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=480&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [commanderijen](https://id.erfgoed.net/thesauri/erfgoedtypes/102) ([13](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=102) erfgoedobjecten, [10](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=102&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [priorijen](https://id.erfgoed.net/thesauri/erfgoedtypes/354) ([19](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=354) erfgoedobjecten, [15](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=354&geldig=1&categorie=beschermingen) aanduidingsobjecten)
* [abdijen](https://id.erfgoed.net/thesauri/erfgoedtypes/4) ([89](https://inventaris.onroerenderfgoed.be/erfgoedobjecten?typologie=4) erfgoedobjecten, [82](https://inventaris.onroerenderfgoed.be/aanduidingsobjecten?typologie=4&geldig=1&categorie=beschermingen) aanduidingsobjecten)
